# Conception et Implémentation d’un Modem QAM en Python

## Objectif du projet

Ce projet a pour but de concevoir et simuler un modem numérique basé sur la modulation d'amplitude en quadrature (QAM), une technique de modulation utilisée dans les systèmes de communication numériques.  
L’objectif est d’implémenter chaque étape du modem (TX/RX), de visualiser la constellation, d’ajouter un codage Gray, de calculer les performances (BER/SNR) et de tester la transmission d’une image.


# PARTIE 1 : Définition des Fonctions de Simulation QAM

Dans cette partie, nous définissons toutes les fonctions nécessaires à la simulation d’un modem QAM.  
Elles sont regroupées en 8 blocs fonctionnels :

1. Génération des codes binaire et Gray  
2. Construction de la constellation QAM avec différentes orientations  
3. Création du tableau de correspondance symbole ↔ coordonnées IQ  
4. Affichage de la constellation TX  
5. Ajout de bruit (AWGN) et de déphasage (bruit de phase)  
6. Démodulation et calcul du taux d’erreur binaire (BER)  
7. Tracé de la courbe BER en fonction du SNR  
8. Transmission et reconstruction d’une image via QAM

Ces fonctions seront utilisées dans la partie suivante pour simuler un système complet.


## 1.Importation des bibliothèques nécessaires

Nous commençons par importer les bibliothèques utiles : calcul numérique avec NumPy, visualisation avec Matplotlib, traitement d’images avec PIL..

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from math import log2, sqrt, pi, atan2
from sklearn.metrics import mean_squared_error
from PIL import Image
import os

## 2.Codage Gray et Binaire

Ces fonctions permettent de :
- Générer les représentations binaires pour une constellation de taille M
- Convertir en code Gray pour minimiser les erreurs de bit entre symboles proches

In [10]:
def gray_code(n):
    return n ^ (n >> 1)

def binary_to_gray_list(M):
    return [format(gray_code(i), f"0{int(log2(M))}b") for i in range(M)]

def binary_list(M):
    return [format(i, f"0{int(log2(M))}b") for i in range(M)]


## 3. Génération Constellation QAM avec orientation personnalisée

Cette fonction crée la constellation QAM selon l’orientation choisie. Chaque point IQ est positionné sur un plan en fonction de son index.


In [13]:
def generate_qam_constellation(M, origin="bottom-left"):
    m_side = int(sqrt(M))
    assert m_side ** 2 == M, "M must be un carré parfait"

    iq_values = []
    for idx in range(M):
        i = idx // m_side  # ligne
        j = idx % m_side   # colonne

        I = 2 * j - (m_side - 1)
        Q = 2 * i - (m_side - 1)

        if origin == "top-left":
            Q = -Q
        elif origin == "top-right":
            Q = -Q
            I = -I
        elif origin == "bottom-right":
            I = -I

        iq_values.append((I, Q))

    return iq_values

## 4. Tableau de correspondance + bilans

Cette fonction :
- Génère un tableau associant chaque symbole à ses coordonnées I/Q, son énergie, et sa phase.
- Affiche un histogramme de la distribution des phases.
- Affiche les bilans statistiques sur l'énergie et la phase des symboles.

In [16]:
def generate_symbol_table(M, use_gray=True, origin="bottom-left", show_hist=True):
    bin_list = binary_to_gray_list(M) if use_gray else binary_list(M)
    iq_vals = generate_qam_constellation(M, origin)

    table = []
    energies = []
    phases = []

    for idx, (I, Q) in enumerate(iq_vals):
        bits = bin_list[idx]
        dec = int(bits, 2)
        energy = I**2 + Q**2
        phase = atan2(Q, I)
        table.append([idx, bits, dec, I, Q, energy, np.degrees(phase)])
        energies.append(energy)
        phases.append(np.degrees(phase))

    df = pd.DataFrame(table, columns=["Symbole", "Binaire", "Décimal", "I", "Q", "Énergie", "Phase (°)"])

    print("\n\033[1mBILAN DES ÉNERGIES ET DES PHASES :\033[0m")
    print(f"Énergie moyenne des symboles : {np.mean(energies):.2f}")
    print(f"Phase moyenne : {np.mean(phases):.2f}°")

    if show_hist:
        plt.hist(phases, bins=12, edgecolor='black')
        plt.title("Distribution des phases des symboles")
        plt.xlabel("Phase (°)")
        plt.ylabel("Nombre de symboles")
        plt.grid(True)
        plt.show()

    return df

## 5. Affichage Constellation TX

Affiche la constellation QAM avec les symboles binaires associés à chaque point (I,Q).  
Utile pour visualiser l’organisation spatiale des symboles avant transmission.

In [19]:
def plot_constellation(df, title="Constellation TX"):
    plt.figure(figsize=(6,6))
    plt.scatter(df['I'], df['Q'], c='blue')
    for i, row in df.iterrows():
        plt.text(row['I']+0.2, row['Q']+0.2, row['Binaire'], fontsize=9)
    plt.axhline(0, color='black')
    plt.axvline(0, color='black')
    plt.grid(True)
    plt.title(title)
    plt.xlabel("I")
    plt.ylabel("Q")
    plt.axis('equal')
    plt.show()

## 6. Ajout de bruit AWGN + déphasage

Ces fonctions simulent les perturbations du canal de transmission :
- Bruit blanc gaussien (AWGN)
- Bruit de phase (variation aléatoire de la phase)

In [22]:
def add_awgn_noise(iq_points, snr_db):
    snr_linear = 10**(snr_db / 10)
    power_signal = np.mean(np.abs(iq_points)**2)
    noise_power = power_signal / snr_linear
    noise = np.sqrt(noise_power / 2) * (np.random.randn(*iq_points.shape) + 1j * np.random.randn(*iq_points.shape))
    return iq_points + noise

def apply_phase_noise(iq_points, phase_std_deg):
    phase_noise = np.radians(np.random.normal(0, phase_std_deg, iq_points.shape[0]))
    return iq_points * np.exp(1j * phase_noise)

## 7. Démodulation & BER

- Démodulation des symboles reçus par recherche du point QAM le plus proche.
- Comparaison des bits transmis et reçus pour calculer le taux d’erreur binaire (BER).

In [25]:
def demodulate(received_points, iq_ref, bits_ref):
    decoded_bits = []
    for r in received_points:
        distances = [np.abs(r - complex(I, Q)) for I, Q in iq_ref]
        index = np.argmin(distances)
        decoded_bits.append(bits_ref[index])
    return decoded_bits

def calculate_ber(bits_sent, bits_received):
    total_bits = len(bits_sent) * len(bits_sent[0])
    errors = sum(b1 != b2 for s, r in zip(bits_sent, bits_received) for b1, b2 in zip(s, r))
    return errors / total_bits

## 8. Courbe BER vs SNR

Cette fonction simule plusieurs transmissions pour différents niveaux de bruit (SNR).
Elle trace la courbe du Bit Error Rate (BER) en fonction du SNR, permettant d’évaluer la performance du système.


In [28]:
def ber_vs_snr(M, use_gray, phase_noise_std, snr_range):
    df_symbols = generate_symbol_table(M, use_gray, show_hist=False)
    iq_ref = list(zip(df_symbols['I'], df_symbols['Q']))
    bits_ref = list(df_symbols['Binaire'])
    iq_sent = np.array([complex(I, Q) for I, Q in iq_ref])
    bits_sent = bits_ref.copy()

    ber_list = []
    for snr_db in snr_range:
        iq_rx = add_awgn_noise(iq_sent, snr_db)
        iq_rx = apply_phase_noise(iq_rx, phase_noise_std)
        bits_received = demodulate(iq_rx, iq_ref, bits_ref)
        ber = calculate_ber(bits_sent, bits_received)
        ber_list.append(ber)

    plt.figure()
    plt.semilogy(snr_range, ber_list, marker='o')
    plt.grid(True, which='both')
    plt.title(f"BER vs SNR (QAM-{M}, Gray={use_gray})")
    plt.xlabel("SNR (dB)")
    plt.ylabel("Bit Error Rate (BER)")
    plt.show()

## 9. Transmission d'image

Ces fonctions permettent :
- De convertir une image en flux binaire
- De la transmettre par QAM avec bruit
- De reconstruire l’image reçue
Elles permettent de visualiser l’impact du bruit sur une image réelle.

In [31]:
def image_to_bits(img_path):
    img = Image.open(img_path).convert('L')
    img = img.resize((64, 64))
    pixels = np.array(img).flatten()
    bits = ''.join([format(p, '08b') for p in pixels])
    return bits, img.size

def bits_to_image(bits, size):
    pixels = [int(bits[i:i+8], 2) for i in range(0, len(bits), 8)]
    img = np.array(pixels, dtype=np.uint8).reshape(size[::-1])
    return Image.fromarray(img, mode='L')

def transmit_image(img_path, M, use_gray, snr_db, phase_noise_std):
    df_symbols = generate_symbol_table(M, use_gray, show_hist=False)
    iq_ref = list(zip(df_symbols['I'], df_symbols['Q']))
    bits_ref = list(df_symbols['Binaire'])
    k = int(log2(M))

    bitstream, img_size = image_to_bits(img_path)
    bitstream_padded = bitstream + '0' * ((k - len(bitstream) % k) % k)

    symbols = [bitstream_padded[i:i+k] for i in range(0, len(bitstream_padded), k)]
    modulated = np.array([complex(*iq_ref[bits_ref.index(b)]) for b in symbols])

    modulated_noisy = apply_phase_noise(add_awgn_noise(modulated, snr_db), phase_noise_std)
    received_symbols = demodulate(modulated_noisy, iq_ref, bits_ref)
    received_bits = ''.join(received_symbols)[:len(bitstream)]

    img_rx = bits_to_image(received_bits, img_size)
    return img_rx

# PARTIE 2 : Simulation d’un Modem QAM avec Entrées Utilisateur

Dans cette partie, nous utilisons les fonctions définies précédemment pour simuler un système de transmission numérique basé sur la modulation QAM.  
L’utilisateur peut spécifier différents paramètres comme :

- Le nombre de symboles (M) de la constellation QAM
- L'utilisation ou non du code Gray
- L’orientation de la constellation
- Le niveau de bruit (SNR)
- Le bruit de phase
- Le chemin de l’image à transmettre (optionnel)

La simulation inclut :
- La génération et l’affichage de la constellation QAM
- Le calcul du BER en fonction du SNR
- La transmission et la reconstruction d’une image

Cette partie permet de valider le bon fonctionnement du système en conditions réalistes de transmission.


In [ ]:
if __name__ == '__main__':
    print("==== SIMULATEUR QAM ====")
    mode = input("Choisissez l'input (1 = Nbre de symboles M, 2 = Nbre de bits/symbole k): ")
    if mode.strip() == '1':
        M = int(input("Entrez le nombre de symboles M (ex: 16, 64): "))
    else:
        k = int(input("Entrez le nombre de bits par symbole k (ex: 4 pour QAM-16): "))
        M = 2 ** k

    use_gray_input = input("Utiliser le code Gray ? (o/n): ").strip().lower()
    use_gray = use_gray_input == 'o'

    print("\nPosition du premier symbole dans la grille:")
    print(" 1 - En bas à gauche (par défaut)")
    print(" 2 - En haut à gauche")
    print(" 3 - En haut à droite")
    print(" 4 - En bas à droite")
    origin_choice = input("Votre choix (1 à 4): ").strip()
    origin_map = {"1": "bottom-left", "2": "top-left", "3": "top-right", "4": "bottom-right"}
    origin = origin_map.get(origin_choice, "bottom-left")

    snr_db = float(input("Valeur du SNR (dB) ? (ex: 10): "))
    phase_noise_std = float(input("Écart type du bruit de phase (en degrés) ? (ex: 5): "))

   
    df_symbols = generate_symbol_table(M, use_gray, origin, show_hist=True)
    print("\nTable de correspondance des symboles:")
    print(df_symbols)
    plot_constellation(df_symbols)

    iq_ref = list(zip(df_symbols['I'], df_symbols['Q']))
    bits_ref = list(df_symbols['Binaire'])
    iq_sent = np.array([complex(I, Q) for I, Q in iq_ref])
    bits_sent = bits_ref.copy()

    iq_rx = add_awgn_noise(iq_sent, snr_db)
    iq_rx = apply_phase_noise(iq_rx, phase_noise_std)
    bits_received = demodulate(iq_rx, iq_ref, bits_ref)
    ber = calculate_ber(bits_sent, bits_received)
    print(f"\nBER (SNR={snr_db} dB, bruit de phase={phase_noise_std}°) : {ber:.5f}")

    plt.figure(figsize=(6,6))
    plt.scatter(iq_rx.real, iq_rx.imag, c='red')
    plt.axhline(0, color='black')
    plt.axvline(0, color='black')
    plt.grid(True)
    plt.title(f"Constellation RX avec bruit (SNR={snr_db} dB)")
    plt.xlabel("I")
    plt.ylabel("Q")
    plt.axis('equal')
    plt.show()

    snr_range = range(0, 21, 2)
    ber_vs_snr(M, use_gray, phase_noise_std, snr_range)

    img_path = r"C:\Users\telephony\Pictures\Screenshots\Capture d'écran 2025-04-23 191128.png"
    if os.path.exists(img_path):
        img_rx = transmit_image(img_path, M, use_gray, snr_db, phase_noise_std)
        plt.figure()
        plt.imshow(img_rx, cmap='gray')
        plt.title("Image reçue après transmission QAM")
        plt.axis('off')
        plt.show()
    else:
        print("\nImage 'test_image.png' non trouvée. Placez une image PNG en niveaux de gris dans le répertoire.")
        


==== SIMULATEUR QAM ====
